# QuickStart

_ActionWeaver is an AI application framework that puts function-calling as a first-class feature_.

---

This demonstrates how to seamlessly integrate any Python function into your LLM application using ActionWeaver and either the Azure or OpenAI API.
 

## Patch OpenAI API

In [1]:
from openai import OpenAI
from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver.llms.openai.tools.tools import Tools
client = OpenAI()
client = OpenAIChatCompletion.patch(client)



In [7]:
from actionweaver import action

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather],
    stream=False
)



Getting current weather
Getting current weather


In [8]:
response

"The current weather in San Francisco is 72°F. However, I'm sorry but I couldn't find the current weather information for Beijing."

In [9]:
def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')


print_output(response)

The current weather in San Francisco is 72°F. However, I'm sorry but I couldn't find the current weather information for Beijing.


## Use ActionWeaver and OpenAI API

In [10]:
import os
import json
import openai
from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver import action

openai.api_key = os.getenv("OPENAI_API_KEY")

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"

print (json.dumps(get_current_time.get_function_details(), indent=4))

{
    "name": "GetCurrentTime",
    "description": "\n    Use this for getting the current time in the specified time zone.\n    \n    :return: A string representing the current time in the specified time zone.\n    ",
    "parameters": {
        "properties": {},
        "title": "Get_Current_Time",
        "type": "object"
    }
}


In [11]:
chat = OpenAIChatCompletion("gpt-3.5-turbo")
messages = [{"role": "user", "content": "what time is it now"}]
chat.create(messages=messages, actions = [get_current_time], stream=True)

Getting current time...


## Use ActionWeaver and Azure OpenAI Service

In [8]:
from actionweaver.llms.azure.chat import ChatCompletion

chat = ChatCompletion(
    model="gpt-35-turbo-0613-16k", 
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview",
)

chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False)

Getting current time...


'The current time is 16:17:35.'

### Force execute an action

In [36]:
get_current_time.invoke(chat, [{"role": "user", "content": "what time"}], stream=False, force=False)

Getting current time...


'The current time is 14:44.'

### Stop the Action in the loop

Every action comes with a stop argument, which is set to False by default, if True this means that the LLM will immediately return the function's output if chosen, but this also restricts the LLM from making multiple function calls. For instance, if asked about the weather in NYC and San Francisco, the model would invoke two separate functions sequentially for each city. However, with `stop=True`, this process is interrupted once the first function returns weather information for either NYC or San Francisco, depending on which city it queries first.

In [8]:
@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [12]:
chat.create([{"role": "user", "content": "What's the weather like in San Francisco, Tokyo?"}], actions = [get_current_weather])

Getting current weather
Getting current weather


'The weather in San Francisco is currently 72 degrees Fahrenheit. In Tokyo, it is 10 degrees Celsius.'

In [13]:
get_current_weather.stop = True

In [14]:
chat.create([{"role": "user", "content": "What's the weather like in San Francisco, Tokyo?"}], actions = [get_current_weather])

Getting current weather


'{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'